##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# SessionRunHook을 Keras 콜백으로 마이그레이션하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/sessionrunhook_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>


TensorFlow 1에서 훈련 동작을 사용자 정의하려면 `tf.estimator.Estimator`와 함께 `tf.estimator.SessionRunHook`를 사용해야 합니다. 이 가이드는 `tf.keras.callbacks.Callback` API를 사용하여 `SessionRunHook`에서 TensorFlow 2의 사용자 정의 콜백으로 마이그레이션하는 방법을 설명합니다. 이 API는 훈련용 Keras `Model.fit`(`Model.evaluate` 및 `Model.predict`도 동일)와 함께 작동합니다. 훈련을 진행하는 동안 초당 예제를 측정하는 `SessionRunHook` 및 `Callback` 작업을 구현하여 이를 수행하는 방법을 배우게 됩니다.

콜백의 예제로는 체크포인트 저장(`tf.keras.callbacks.ModelCheckpoint`) 및 [TensorBoard](%60tf.keras.callbacks.TensorBoard%60) 요약문 작성이 있습니다. Keras [콜백](https://www.tensorflow.org/guide/keras/custom_callback)은 내장된 Keras `Model.fit`/`Model.evaluate`/`Model.predict` API에서 훈련/평가/예측을 진행하는 동안 서로 다른 지점에서 호출되는 객체입니다. 콜백에 대한 자세한 내용은 `tf.keras.callbacks.Callback` API 문서와 [직접 콜백 작성하기](https://www.tensorflow.org/guide/keras/custom_callback.ipynb/) 및 [내장 메서드를 사용하여 훈련하고 평가하기](https://www.tensorflow.org/guide/keras/train_and_evaluate)(*콜백 사용하기* 섹션) 가이드를 참조하세요.

## 설치하기

데모를 위해 가져오기 및 간단한 데이터세트로 시작해 보겠습니다.

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

import time
from datetime import datetime
from absl import flags

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1: tf.estimator API로 사용자 정의 SessionRunHook 생성하기

다음 TensorFlow 1 예제는 훈련을 진행하는 동안 초당 예제를 측정하는 사용자 정의 `SessionRunHook`를 설정하는 방법을 보여줍니다. 후크(`LoggerHook`)를 생성한 후 `tf.estimator.Estimator.train`의 `hooks` 매개변수에 전달합니다.

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (features, labels)).batch(1).repeat(100)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [ ]:
class LoggerHook(tf1.train.SessionRunHook):
  """Logs loss and runtime."""

  def begin(self):
    self._step = -1
    self._start_time = time.time()
    self.log_frequency = 10

  def before_run(self, run_context):
    self._step += 1

  def after_run(self, run_context, run_values):
    if self._step % self.log_frequency == 0:
      current_time = time.time()
      duration = current_time - self._start_time
      self._start_time = current_time
      examples_per_sec = self.log_frequency / duration
      print('Time:', datetime.now(), ', Step #:', self._step,
            ', Examples per second:', examples_per_sec)

estimator = tf1.estimator.Estimator(model_fn=_model_fn)

# Begin training.
estimator.train(_input_fn, hooks=[LoggerHook()])

## TensorFlow 2: Model.fit용 사용자 정의 Keras 콜백 생성하기

TensorFlow 2에서 훈련/평가에 내장 Keras `Model.fit`(또는 `Model.evaluate`)를 사용하는 경우 사용자 정의`tf. keras.callbacks.Callback`을 구성한 후 이를 `Model.fit`(또는 `Model.evaluate`)의 `callbacks` 매개변수로 전달합니다(자세한 내용은 [직접 콜백 작성하기](../..guide/keras/custom_callback.ipynb) 가이드에서 확인).

아래 예제에서는 다양한 메트릭을 기록하는 사용자 정의 `tf.keras.callbacks.Callback`을 작성합니다. 초당 예제를 측정하며 이전 `SessionRunHook`의 메트릭과 비교할 수 있어야 합니다.

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):

    def on_train_begin(self, logs = None):
      self._step = -1
      self._start_time = time.time()
      self.log_frequency = 10

    def on_train_batch_begin(self, batch, logs = None):
      self._step += 1

    def on_train_batch_end(self, batch, logs = None):
      if self._step % self.log_frequency == 0:
        current_time = time.time()
        duration = current_time - self._start_time
        self._start_time = current_time
        examples_per_sec = self.log_frequency / duration
        print('Time:', datetime.now(), ', Step #:', self._step,
              ', Examples per second:', examples_per_sec)

callback = CustomCallback()

dataset = tf.data.Dataset.from_tensor_slices(
    (features, labels)).batch(1).repeat(100)

model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer, "mse")

# Begin training.
result = model.fit(dataset, callbacks=[callback], verbose = 0)
# Provide the results of training metrics.
result.history

## 다음 단계

콜백에 대한 자세한 내용:

- API 문서: `tf.keras.callbacks.Callback`
- 가이드: [직접 콜백 작성하기](../..guide/keras/custom_callback.ipynb/)
- 가이드: [내장 메서드를 사용하여 훈련하고 평가하기](https://www.tensorflow.org/guide/keras/train_and_evaluate)(*콜백 사용하기* 섹션)

다음과 같은 마이그레이션 관련 리소스도 유용할 수 있습니다.

- [조기 중단 마이그레이션 가이드](early_stopping.ipynb): 내장 조기 중단 콜백인 `tf.keras.callbacks.EarlyStopping`
- [TensorBoard 마이그레이션 가이드](tensorboard.ipynb): TensorBoard를 사용하여 메트릭 추적 및 표시하기
- [LoggingTensorHook 및 StopAtStepHook에서 Keras 콜백으로의 마이그레이션 가이드](logging_stop_hook.ipynb)